In [ ]:
import csv
import happybase
import time
import sys
import pandas as pd
import numpy
import json

In [ ]:
#Initializition for connecting to hbase using thrift server
batch_size = 1
host = "0.0.0.0"
file_path = "/home/user1/Downloads/rest.csv"
namespace = "default"
row_count = 0
start_time = time.time()
table_name = "REST"

In [ ]:
def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    conn = happybase.Connection(host = host,
        table_prefix = namespace,
        table_prefix_separator = ":",autoconnect=True )
    #conn.open()
    table = conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return conn,batch

In [ ]:
def insert_row(batch, row):
    """ Insert a row into HBase.
    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.
    Rows have the following schema:
        [ id	typ	time	intent	entity	city	state	country	restname	paytype	year	month	date]
    """
    batch.put(row[0], { "data:typ": row[1], "data:time": row[2], "data:intent": row[3],
        "data:entity": row[4], "data:city": row[5], "data:state": row[6],"data:country": row[7],"data:restname": row[8], 
                  "data:paytype": row[9],"data:year": row[10],"data:month": row[11],"data:date": row[12]})

In [ ]:
def read_csv():
    csvfile = open(file_path, "r")
    csvreader = csv.reader(csvfile)
    return csvreader, csvfile

In [ ]:
conn, batch = connect_to_hbase()
print ("Connect to HBase. table name: %s, batch size: %i" % (table_name, batch_size))

In [ ]:
csvreader, csvfile = read_csv()
print ("Connected to file. name: %s" % (file_path))

In [ ]:
try:
    # Loop through the rows. The first row contains column headers, so skip that
    # row. Insert all remaining rows into the database.
    for row in csvreader:
        row_count += 1
        if row_count == 1:
            pass
        else:
            insert_row(batch, row)

    # If there are any leftover rows in the batch, send them now.
    batch.send()
finally:
    # No matter what happens, close the file handle.
    csvfile.close()
    #conn.close()

duration = time.time() - start_time
print ("Done. row count: %i, duration: %.3f s" % (row_count, duration))

In [ ]:
# Reading from Hbase using the connection 
conn, batch = connect_to_hbase()
table = conn.table('REST')
rows = table.scan(columns=[b'data'])
#Intialize df as dataframe
df = pd.DataFrame()
print(df)
for row in rows:
    df_row = {key.decode('utf8').split(':')[1]: value.decode('utf8') for key, value in row[1].items()}
    df = df.append(df_row, ignore_index=True)
type(df)

In [ ]:
#Groupby the desired columns and convert it to csv to do some preprocessing before converting to json
df1=df.groupby(['typ','intent','month']).size()
df3=df1.to_csv()
df3

In [ ]:
#Same preprocessing to rename it as tp convert it to json
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
TESTDATA = StringIO(df3)
df4 = pd.read_csv(TESTDATA, sep=",",header=None)
df4.rename(columns={0: 'typ', 1: 'intent',2: 'month',3: 'count'},inplace=True)


In [ ]:
#https://stackoverflow.com/questions/19798112/convert-pandas-dataframe-to-a-nested-dict
#This is recursive function to convert pandas dataframe to nested dictionary of any level
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.ix[:,1:]) for k,g in grouped}
    return d

In [ ]:
j1=recur_dictify(df4)

In [ ]:
#Function to convert numpy int64 data type to int data type. This is needed as json dumps can 
#work only on integer data types
import numpy
def default(o):
    if isinstance(o, numpy.int64): return int(o)  
    raise TypeError

x6=json.dumps(j1, default=default)